### Markov Clustering을 활용한 분석

In [ ]:
# 필요한 라이브러리 불러오기
import folium
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# 데이터 불러오기
data = pd.read_csv('check1_운행.csv', index_col=0)

# 데이터에 그리드 번호 부여 : 서울을 포함하는 20x20 그리드
# 중심좌표(center) : 서울광장
center = [37.565960761761, 126.97908777727]

# gird 번호 부여하는 함수
def aloc_grid(coord):
    xw = (coord[0] - center[0]) // 0.03
    yw = (coord[1] - center[1]) // 0.038
    if xw < -10 or xw > 9 or yw < -10 or yw > 9:
        ans = 0
    else:
        ans = xw + 20*yw + 211
    return 'grid_{}'.format(int(ans))

# 각 데이터의 출발, 도착좌표에 따라 그리드번호 부여
for idx, srs in data.iterrows():
    data.at[idx, 'grid_x'] = aloc_grid([srs['s_lat'], srs['s_lng']])
    data.at[idx, 'grid_y'] = aloc_grid([srs['e_lat'], srs['e_lng']])
    
# grid 범위 내 데이터만 추출
ins = data[(data['grid_x'] != 'grid_0') & (data['grid_y'] != 'grid_0')]

In [ ]:
# 마르코프 행렬 생성

# 전이행렬 _ 각 grid의 연결횟수를 기록, 연결의 강력함을 볼 것임
# 일단 Transpose로 제작, 마지막에 뒤집을거임
grid_sz = 400

# markov[행, 열]에 데이터 가산
markov = np.zeros((grid_sz, grid_sz))
for idx, srs in ins.iterrows():
    markov[int(srs['grid_x'][5:])-1, int(srs['grid_y'][5:])-1] += 1

# 대각성분을 모두 1로 치환
for i in range(grid_sz):
    markov[i, i] = 1

# 각 행별로 성분을 표준화
for i in range(grid_sz):
    markov[i] = markov[i] / markov[i].sum()

# 행렬을 뒤집기
markov = markov.T

In [ ]:
# 시각화를 위한 격자점 생성
l1 = []
for i in range(20):
    for _ in range(20):
        l1.append(i+1)
l1 = np.array(l1)

l2 = []
for _ in range(20):
    for i in range(20):
        l2.append(i+1)
l2 = np.array(l2)


# 시각화 _ 각 그리드를 잇는 성분값이 클수록, 해당 그리드를 잇는 선이 짙어짐
# 1차 시각화
fig = plt.figure(figsize = (10, 10))

for i in range(grid_sz):
    for j in range(grid_sz):
        if markov[j][i] != 0:
            plt.plot([i//20 +1, j//20 +1], [i+1 - 20*(i//20), j+1 - 20*(j//20)], linewidth=markov[j, i]/np.max(markov)*5, alpha=0.3, color = 'red')
    
plt.scatter(l1, l2, s=50)
plt.xticks(range(1, 21))
plt.yticks(range(1, 21))

plt.show()

In [ ]:
# n차 이상의 연산을 위해 함수화
# matrix : 초기 markov 행렬 : 가산 ~ 뒤집기까지 완료된 행렬
# grid_num : 그리드 개수
# level : 반복연산의 횟수
def mk_view(matrix, grid_num, level):
    for _ in range(level):
        matrix_T = ((np.matmul(matrix, matrix))**2).T
        for i in range(grid_num):
            matrix_T[i] = matrix_T[i] / matrix_T[i].sum()
        matrix = matrix_T.T
    
    fig = plt.figure(figsize = (10, 10))
    for i in range(grid_num):
        for j in range(grid_num):
            if matrix[j][i] != 0:
                plt.plot([i//20 +1, j//20 +1], [i+1 - 20*(i//20), j+1 - 20*(j//20)], linewidth=matrix[j, i]/np.max(matrix)*5, alpha=0.3, color = 'red')
    plt.scatter(l1, l2, s=50)
    plt.xticks(range(1, 21))
    plt.yticks(range(1, 21))
    plt.show()

    return matrix

re_mat = mk_view(markov, grid_sz, 2)

In [ ]:
# 연결성이 가장 높은 그리드 6개 선택
grid_hap = (re_mat + re_mat.T).sum(axis = 0)
idx = (-grid_hap).argsort()[:6]

for i in range(6):
  globals()['c{}'.format(i+1)] = ins[ins['grid_x'] == 'grid_{}'.format(idx[i]+1)]
  
# 각 중심점별로, 중심점이 출발일 때의 도착점 시각화
end = folium.Map(location=[37.53897093698831, 127.05461953077439], zoom_start=14)

for i in range(len(c1)):
  folium.CircleMarker(location = [c1.iloc[i]['e_lat'], c1.iloc[i]['e_lng']] , radius = 4, color='red').add_to(end)  
for i in range(len(c6)):
  folium.CircleMarker(location = [c6.iloc[i]['e_lat'], c6.iloc[i]['e_lng']] , radius = 4, color='blue').add_to(end)
for i in range(len(c2)):
  folium.CircleMarker(location = [c2.iloc[i]['e_lat'], c2.iloc[i]['e_lng']] , radius = 4, color='yellow').add_to(end) 
for i in range(len(c3)):
  folium.CircleMarker(location = [c3.iloc[i]['e_lat'], c3.iloc[i]['e_lng']] , radius = 4, color='black').add_to(end) 
for i in range(len(c4)):
  folium.CircleMarker(location = [c4.iloc[i]['e_lat'], c4.iloc[i]['e_lng']] , radius = 4, color='pink').add_to(end) 
for i in range(len(c5)):
  folium.CircleMarker(location = [c5.iloc[i]['e_lat'], c5.iloc[i]['e_lng']] , radius = 4, color='green').add_to(end)


# 각 중심점별로 출발점 시각화
start = folium.Map(location=[37.53897093698831, 127.05461953077439], zoom_start=14)

for i in range(len(c1)):
  folium.CircleMarker(location = [c1.iloc[i]['s_lat'], c1.iloc[i]['s_lng']] , radius = 4, color='red').add_to(start)  
for i in range(len(c6)):
  folium.CircleMarker(location = [c6.iloc[i]['s_lat'], c6.iloc[i]['s_lng']] , radius = 4, color='blue').add_to(start)
for i in range(len(c2)):
  folium.CircleMarker(location = [c2.iloc[i]['s_lat'], c2.iloc[i]['s_lng']] , radius = 4, color='yellow').add_to(start) 
for i in range(len(c3)):
  folium.CircleMarker(location = [c3.iloc[i]['s_lat'], c3.iloc[i]['s_lng']] , radius = 4, color='black').add_to(start) 
for i in range(len(c4)):
  folium.CircleMarker(location = [c4.iloc[i]['s_lat'], c4.iloc[i]['s_lng']] , radius = 4, color='pink').add_to(start) 
for i in range(len(c5)):
  folium.CircleMarker(location = [c5.iloc[i]['s_lat'], c5.iloc[i]['s_lng']] , radius = 4, color='green').add_to(start)

In [ ]:
end.save('클러스터링 도착점.html')
start.save('클러스터링 중심점.html')